In [1]:
####################################################
# Base de dados Abertas
####################################################

# GOV BR
# https://dados.gov.br/dados/conjuntos-dados

# BASE DOS DADOS .ORG
# https://basedosdados.org/

# Estatísticas IBGE
# https://www.ibge.gov.br/estatisticas/downloads-estatisticas.html

# Portal da Transparência
# https://portaldatransparencia.gov.br/download-de-dados

# Dados Abertos do Banco Central do Brasil
# https://dadosabertos.bcb.gov.br/

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum, avg
import time
import pandas as pd

In [6]:
# Inicializa a sessão Spark
spark = SparkSession.builder.appName("BaseDadosAbertas").getOrCreate()

# nome_arquivo_csv = "br_rf_arrecadacao_uf.csv"
nome_arquivo_csv = "br_rf_arrecadacao_uf_maior.csv"

# carregar dados
df = spark.read.csv(nome_arquivo_csv, header=True, inferSchema=True)

# Operações básicas
filtered_df = df.filter(col("ipi_automoveis") > 10000)
grouped_df = df.groupBy("sigla_uf").agg(sum("irpf")).alias("total_irpf")

# Operação complexa para comparação de desempenho
def complex_operation(df):
    start_time = time.time()
    result = df.groupBy("ano", "sigla_uf") \
        .agg(sum("irpf").alias("total_irpf"), avg("ipi_fumo").alias("media_ipi_fumo")) \
        .orderBy("ano", "sigla_uf")
    result.count() # Forçar a execução da operação
    end_time = time.time()
    return end_time - start_time

spark_time = complex_operation(df)
print(f"Tempo de execução com Spark: {spark_time} segundos")

# Comparação com Pandas (se o conjunto de dados for pequeno o suficiente para caber na memória)
pandas_df = df.toPandas()
# pandas_df = pd.read_csv(nome_arquivo_csv)

def pandas_complex_operation(df):
    start_time = time.time()
    result = df.groupby(["ano", "sigla_uf"]) \
        .agg({"irpf": "sum", "ipi_fumo": "mean"}) \
        .reset_index() \
        .sort_values(by=["ano", "sigla_uf"])
    end_time = time.time()
    return end_time - start_time
# def pandas_complex_operation(df):
#     start_time = time.time()
#     result = df.groupby(["ano", "sigla_uf"]).agg(
#         irpf_sum=pd.NamedAgg(column="irpf", aggfunc="sum"),
#         ipi_fumo_mean=pd.NamedAgg(column="ipi_fumo", aggfunc="mean")
#     ).reset_index().sort_values(by=["ano", "sigla_uf"])
#     end_time = time.time()
#     return end_time - start_time

pandas_time = pandas_complex_operation(pandas_df)
print(f"Tempo de execução com Pandas: {pandas_time} segundos")

# Visualizar planos de execução
df.groupBy("ano").agg(sum("irpf")).alias("total_irpf").explain()

# Encerrar a sessão Spark
spark.stop()

Tempo de execução com Spark: 0.5313839912414551 segundos


24/11/24 11:40:55 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


Tempo de execução com Pandas: 0.03389120101928711 segundos
== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- HashAggregate(keys=[ano#833], functions=[sum(irpf#843)])
   +- Exchange hashpartitioning(ano#833, 200), ENSURE_REQUIREMENTS, [plan_id=469]
      +- HashAggregate(keys=[ano#833], functions=[partial_sum(irpf#843)])
         +- FileScan csv [ano#833,irpf#843] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/Users/valter/Documents/dev/Estudos/Pos-IA-ML/UC9_Linguagens_de_p..., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<ano:int,irpf:double>


